In [1]:
!pip install keras-tuner -q

     |████████████████████████████████| 71kB 3.1MB/s 


In [40]:
import copy
import os

import tensorflow as tf

from kerastuner.engine import base_tuner

class DeepTuner(base_tuner.BaseTuner):

    def run_trial(self, trial, x, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)
        model.fit(x, y, validation_data=validation_data,
                  batch_size=trial.hyperparameters.Choice("batch_size", [16, 32]),
                  **fit_kwargs)

        x_val, y_val = validation_data  # get the validation data
        self.oracle.update_trial(trial.trial_id,
                                 {name: value for name, value in zip(
                                     model.metrics_names, 
                                     model.evaluate(x_val, y_val))})  # inform the oracle of the eval result, the result is a dictionary with the metric names as the keys.
        self.save_model(trial.trial_id, model) # save the model to disk

    def save_model(self, trial_id, model):
        fname = os.path.join(self.get_trial_dir(trial_id), 'model.pickle')
        model.save(fname)

    def load_model(self, trial):
        fname = os.path.join(self.get_trial_dir(trial.trial_id), 'model.pickle')
        model = tf.keras.models.load_model(fname)
        return model



In [28]:
from sklearn.datasets import load_digits

# Load the hand-written digits dataset
digits = load_digits()

# Get the images and corresponding labels
images, labels = digits.images, digits.target
images.shape, labels.shape

# reshape images to vectors
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))

# Split data into train and test subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, shuffle=False)

In [32]:
import numpy as np

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [42]:
import kerastuner as kt

def build_model(hp):
    model = tf.keras.Sequential(
        [
            tf.keras.Input(shape=input_shape),
            tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Conv2D(hp.Choice("filters", [64, 128, 256]), kernel_size=(3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ])
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

random_tuner = DeepTuner(
        oracle=kt.oracles.RandomSearch(
            objective=kt.Objective('accuracy', 'max'),
            max_trials=3,
            seed=42),
        hypermodel=build_model,
        overwrite=True,
        project_name='random_tuner')

random_tuner.search(x_train[:10], y_train[:10], validation_data=(x_test, y_test))
random_tuner.search_space_summary()

Trial 3 Complete [00h 00m 09s]
accuracy: 0.1615000069141388

Best accuracy So Far: 0.20149999856948853
Total elapsed time: 00h 00m 29s
INFO:tensorflow:Oracle triggered exit
Search space summary
Default search space size: 2
filters (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 256], 'ordered': True}
batch_size (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32], 'ordered': True}


In [44]:
best_model = random_tuner.get_best_models(1)[0]
y_pred_test = best_model.evaluate(x_test, y_test)
print(y_pred_test)

313/313 [==============================] - 3s 9ms/step - loss: 2.2968 - accuracy: 0.2015
[2.2968461513519287, 0.20149999856948853]
